In [1]:
wdir='./' ## working dir
gColab=True  ## set this to False if running locally
if(gColab):
    ## Mount google drive to the machine running this nb
    from google.colab import drive
    drive.mount('/content/drive')
    ## must also specify the absolute path to the working dir
    wdir='/content/drive/MyDrive/ysNN/partC/'

Mounted at /content/drive


In [2]:
import numpy as np
from tensorflow import keras
#from tensorflow import float64 as tfloat64
import tensorflow as tf
import os,sys
sys.path.append(os.path.abspath(wdir))

#tf.keras.backend.set_floatx('float64') ## cannot do this on float32 trained NNs
nb='nb14'
wFile=wdir+nb+'_fitPolyN8'
dirValues=nb+'_dirValues.txt'
hessValues=nb+'_hessValues.txt'
gradValues=nb+'_gradValues.txt'
nnFile=nb+'_NNdata.txt'

In [5]:
#### Define model
def pwAct10(x):
    return x**10

nnHom=10
nun1=27
model=keras.Sequential()
model.add(keras.layers.Dense(nun1, input_shape=(3,), activation=pwAct10, use_bias=False))
model.add(keras.layers.Dense(1, use_bias=False))

model.load_weights(wFile, skip_mismatch=False, by_name=False, options=None)

In [6]:
### Info on weights
model.summary()
ww=model.get_weights()
print(f'Number of layers = {len(ww)}')
for kk in range(len(ww)):
    print(f'shape of layer {kk}:  {ww[kk].shape}; dtype: {ww[kk].dtype}')

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 27)                81        
                                                                 
 dense_3 (Dense)             (None, 1)                 27        
                                                                 
Total params: 108 (432.00 Byte)
Trainable params: 108 (432.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Number of layers = 2
shape of layer 0:  (3, 27); dtype: float32
shape of layer 1:  (27, 1); dtype: float32


In [7]:
### test NN-function
def nnFuncD(x,ww):
    va=pwAct10(np.dot(x,ww[0].astype(np.double)))
    return np.dot(va,ww[1].astype(np.double))
def nnFunc(x,ww):
    va=pwAct10(np.dot(x,ww[0]))
    return np.dot(va,ww[1])

In [8]:
### Test values (directional properties)
Nphi=21
vphi=np.linspace(0,np.pi/2,Nphi);vDegs=180.0*(vphi/np.pi)
vcos=np.cos(vphi);vsin=np.sin(vphi)
vcos2,vsin2,vsc=vcos*vcos,vsin*vsin,vcos*vsin
vSnn,vRnn , vTest = np.zeros(Nphi),np.zeros(Nphi), np.zeros(Nphi)
for k in range(Nphi):
    vsx=vcos2[k]
    vsy=vsin2[k]
    vsxy=vsc[k]
    vp=np.array([vsx,vsy,vsxy]).reshape((1,3))
    #x = tf.convert_to_tensor(vp, dtype=tf.float64)
    x = tf.convert_to_tensor(vp, dtype=tf.float32)
    with tf.GradientTape() as tape:
        tape.watch(x)
        yf = model(x)
    vSnn[k]=1.0/yf**(1.0/nnHom)
    ##print(tape.gradient(yf, x))
    vDX,vDY,vDXY = tape.gradient(yf, x)[0]
    vRnn[k]=(vDXY*vsc[k]-vDX*vsin2[k]-vDY*vcos2[k])/(vDX+vDY)
    vTest[k]=1.0/(nnFunc(vp,ww))**(1.0/nnHom)
print('row,      sigma,            test_Sigma,          r-Value')
with open(wdir+dirValues,'w') as ff:
    for kk in range(Nphi):
        ff.write(f'{vSnn[kk]}, {vRnn[kk]}\n')
        print(kk, ', ', vSnn[kk],', ', vTest[kk],',', vRnn[kk])

row,      sigma,            test_Sigma,          r-Value
0 ,  0.998661994934082 ,  0.9986620469479156 , 0.542937159538269
1 ,  0.9925248026847839 ,  0.9925248081226741 , 0.5110132694244385
2 ,  0.9761003255844116 ,  0.9761002972153179 , 0.4471372663974762
3 ,  0.9562944173812866 ,  0.9562943355596867 , 0.391547828912735
4 ,  0.9383764863014221 ,  0.9383764751223759 , 0.3570767343044281
5 ,  0.9247057437896729 ,  0.9247057501293391 , 0.34032970666885376
6 ,  0.9158976078033447 ,  0.9158975602142173 , 0.33109864592552185
7 ,  0.9115100502967834 ,  0.9115100145571114 , 0.3194987177848816
8 ,  0.9101457595825195 ,  0.9101457283097549 , 0.3032722473144531
9 ,  0.9097719788551331 ,  0.9097719476798354 , 0.2885448932647705
10 ,  0.908591091632843 ,  0.9085911211814381 , 0.28212225437164307
11 ,  0.905894935131073 ,  0.9058950027623249 , 0.28504717350006104
12 ,  0.9022640585899353 ,  0.9022640597087224 , 0.29448556900024414
13 ,  0.8992982506752014 ,  0.8992981796577079 , 0.3075403869152069
1

In [9]:
### Raw (yield function) Gradient and Hessian test values

#### sigma = [x1,x2,x3] = [vcos2,vsin2,vsc]
x1=tf.Variable(vcos2[:],name='x1',trainable=False)
x2=tf.Variable(vsin2[:],name='x2',trainable=False)
x3=tf.Variable(vsc[:],name='x3',trainable=False)
g1,g2,g3=[],[],[]
with tf.GradientTape(persistent=True) as tape2:
    tape2.watch([x1,x2,x3])
    with tf.GradientTape() as tape1:
        tape1.watch([x1,x2,x3])
        vy=(model(tf.stack([x1,x2,x3],axis=1)))**(1/nnHom)
    g1,g2,g3=tape1.gradient(vy,[x1,x2,x3])

h11=tape2.gradient(g1,x1,unconnected_gradients=tf.UnconnectedGradients.ZERO).numpy()
h12=tape2.gradient(g1,x2,unconnected_gradients=tf.UnconnectedGradients.ZERO).numpy()
h13=tape2.gradient(g1,x3,unconnected_gradients=tf.UnconnectedGradients.ZERO).numpy()
h22=tape2.gradient(g2,x2,unconnected_gradients=tf.UnconnectedGradients.ZERO).numpy()
h23=tape2.gradient(g2,x3,unconnected_gradients=tf.UnconnectedGradients.ZERO).numpy()
h33=tape2.gradient(g3,x3,unconnected_gradients=tf.UnconnectedGradients.ZERO).numpy()
print('row,     H11,                 H12,                   H13,                   H22,                    H23,                     H33')
with open(wdir+hessValues,'w') as ff:
    for kk in range(Nphi):
        ff.write(f'{h11[kk]}, {h12[kk]}, {h13[kk]}, {h22[kk]}, {h23[kk]}, {h33[kk]}\n')
        print(kk,',',h11[kk],',',h12[kk],',',h13[kk],',',h22[kk],',',h23[kk],',',h33[kk])

row,     H11,                 H12,                   H13,                   H22,                    H23,                     H33
0 , -6.034463240212062e-08 , -1.4481891241757694e-08 , -4.630262395721729e-08 , 2.0260367393493652 , -0.0035335065331310034 , 4.938387870788574
1 , 0.02721630036830902 , -0.04861762002110481 , -0.3419818878173828 , 1.9694150686264038 , 0.4627480208873749 , 4.308876037597656
2 , 0.0765635073184967 , -0.1267884224653244 , -0.46332135796546936 , 1.7645856142044067 , 0.521027684211731 , 2.8427746295928955
3 , 0.11683738976716995 , -0.1637014001607895 , -0.4473538100719452 , 1.4610967636108398 , 0.33108648657798767 , 1.783882737159729
4 , 0.1610105037689209 , -0.16862541437149048 , -0.4407547116279602 , 1.1715407371520996 , 0.13831909000873566 , 1.3115566968917847
5 , 0.22481301426887512 , -0.17481936514377594 , -0.470337837934494 , 0.9721572995185852 , 0.019370418041944504 , 1.1274688243865967
6 , 0.3151376247406006 , -0.21492315828800201 , -0.5089819431304932 , 

In [11]:
print('row,      Grad_1,               Grad_2,               Grad_3')
vg1,vg2,vg3=g1.numpy(),g2.numpy(),g3.numpy()
ff=open(wdir+gradValues,'w')
for kk in range(Nphi):
    ff.write(f'{vg1[kk], {vg2[kk]}, vg3[kk]}\n')
    print(kk,', ', vg1[kk], ', ', vg2[kk], ', ', vg3[kk])
ff.close()

row,      Grad_1,               Grad_2,               Grad_3
0 ,  1.0013396739959717 ,  -0.3523569107055664 ,  -0.007180295884609222
1 ,  0.9867804050445557 ,  -0.3199884593486786 ,  0.36814630031585693
2 ,  0.951901912689209 ,  -0.243962824344635 ,  0.6591739058494568
3 ,  0.9111601710319519 ,  -0.15969249606132507 ,  0.8498019576072693
4 ,  0.8691255450248718 ,  -0.08385641872882843 ,  0.9784067869186401
5 ,  0.8236348628997803 ,  -0.016799772158265114 ,  1.0772602558135986
6 ,  0.7712591290473938 ,  0.04898453876376152 ,  1.1605002880096436
7 ,  0.7075099945068359 ,  0.1239275336265564 ,  1.232070803642273
8 ,  0.6270323395729065 ,  0.2160188853740692 ,  1.2905532121658325
9 ,  0.52757328748703 ,  0.32546383142471313 ,  1.330074667930603
10 ,  0.41430747509002686 ,  0.4441169798374176 ,  1.3427858352661133
11 ,  0.29829084873199463 ,  0.5607286691665649 ,  1.3239877223968506
12 ,  0.1908877193927765 ,  0.6653003096580505 ,  1.2763242721557617
13 ,  0.09990930557250977 ,  0.750525891

In [12]:
### Export the NN details and weights (column-wise)
vdeg=[10]   ####    homogeneity degrees of the activation functions
nLay=len(ww)  ### number of layers
ff=open(wdir+nnFile,'w')
ff.write(f'{nLay}\n')  ###   the (total) number of layers
for kk in range(nLay-1): ### only the hidden layers
    ff.write(f'{ww[kk].shape[1]}\n') ###  number of units in layer
    ff.write(f'{vdeg[kk]}\n')  ####   homogeneity degree
for kk in range (nLay):
    for jj in range(ww[kk].shape[1]):
        for ii in range(ww[kk].shape[0]):
            ff.write(f'{ww[kk][ii,jj]}\n')   #### weight
ff.close()

In [13]:
!jupyter nbconvert --to html  {wdir+nb+'_ExportWeights.ipynb'}

[NbConvertApp] Converting notebook /content/drive/MyDrive/ysNN/partC/nb14_ExportWeights.ipynb to html
[NbConvertApp] Writing 622948 bytes to /content/drive/MyDrive/ysNN/partC/nb14_ExportWeights.html
